In [ ]:
import numpy             as np, pandas as pd 
import matplotlib.pyplot as plt
import seaborn  as sns
import lightgbm as lgb
from   sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn           as sns
from   sklearn.metrics   import mean_squared_error
import utils
from   sklearn           import metrics
import matplotlib
from   woe               import WOE
import matplotlib.pyplot as plt
import seaborn           as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)

In [ ]:
path        = '../data/input/input_pkl/train/'
train       = utils.read_pickles(path)

In [ ]:
path        = '../data/input/input_pkl/test/'
test        = utils.read_pickles(path)

In [ ]:
target = train.target.values
train.drop('target', axis=1, inplace=True)
train.shape, target.shape, test.shape

In [ ]:
pos_idx = (target == 1)
neg_idx = (target == 0)
stats = []
for col in train.columns:
    stats.append([
        train.loc[pos_idx, col].mean(),
        train.loc[pos_idx, col].std(),
        train.loc[neg_idx, col].mean(),
        train.loc[neg_idx, col].std()
    ])
    
stats_df = pd.DataFrame(stats, columns=['pos_mean', 'pos_sd', 'neg_mean', 'neg_sd'])
stats_df.head()

In [ ]:
# priori probability
ppos = pos_idx.sum() / len(pos_idx)
pneg = neg_idx.sum() / len(neg_idx)

def get_proba(x):
    # we use odds P(target=1|X=x)/P(target=0|X=x)
    return (ppos * norm.pdf(x, loc=stats_df.pos_mean, scale=stats_df.pos_sd).prod()) /\
           (pneg * norm.pdf(x, loc=stats_df.neg_mean, scale=stats_df.neg_sd).prod())

In [ ]:
tr_pred = train.apply(get_proba, axis=1)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(target, tr_pred)

In [ ]:
from scipy.stats.kde import gaussian_kde

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
sns.distplot(train.loc[pos_idx, 'var_0'])
kde = gaussian_kde(train.loc[pos_idx, 'var_0'].values)
plt.plot(np.linspace(0, 20), kde(np.linspace(0, 20)))
plt.title('target==1')
plt.subplot(1, 2, 2)
sns.distplot(train.loc[neg_idx, 'var_0'])
kde = gaussian_kde(train.loc[neg_idx, 'var_0'].values)
plt.plot(np.linspace(0, 20), kde(np.linspace(0, 20)))
plt.title('target==0')

In [ ]:
stats_df['pos_kde'] = None
stats_df['neg_kde'] = None
for i, col in enumerate(train.columns):
    stats_df.loc[i, 'pos_kde'] = gaussian_kde(train.loc[pos_idx, col].values)
    stats_df.loc[i, 'neg_kde'] = gaussian_kde(train.loc[neg_idx, col].values)

In [ ]:
def get_proba2(x):
    proba = ppos / pneg
    for i in range(200):
        proba *= stats_df.loc[i, 'pos_kde'](x[i]) / stats_df.loc[i, 'neg_kde'](x[i])
    return proba

In [ ]:
slow, we can speed up by binize the variable values.

def get_col_prob(df, coli, bin_num=100):
    bins = pd.cut(df.iloc[:, coli].values, bins=bin_num)
    uniq = bins.unique()
    uniq_mid = uniq.map(lambda x: (x.left + x.right) / 2)
    dense = pd.DataFrame({
        'pos': stats_df.loc[coli, 'pos_kde'](uniq_mid),
        'neg': stats_df.loc[coli, 'neg_kde'](uniq_mid)
    }, index=uniq)
    return bins.map(dense.pos).astype(float) / bins.map(dense.neg).astype(float)

In [ ]:
tr_pred = ppos / pneg
for i in range(200):
    tr_pred *= get_col_prob(train, i)

In [ ]:
roc_auc_score(target, tr_pred)

In [ ]:
te_pred = ppos / pneg
for i in range(200):
    te_pred *= get_col_prob(test, i)

In [ ]:
pd.DataFrame({
    'ID_code': test.index,
    'target': te_pred
}).to_csv('sub.csv', index=False)

In [ ]:
Conclusion
In this kernel we demonstrate how Naive bayes works, we build Gaussian Naive Bayes, which gives us 0.890 AUC. By remove Gaussian constrain and choosing more accurate kernel function, we can get better performance.

Holp this can help, thanks!